In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from statistics import mode
from matplotlib.colors import LogNorm
from astropy.table import Table
from exod.utils.plotting import use_scienceplots
use_scienceplots()

In [ ]:
evt = Table.read('0783080101_PN.fits')
evt['t0'] = evt['TIME'] - evt['TIME'].min()

In [ ]:
xpos = 24473.458
ypos = 20137.664
size = 325.10496

t_start = 607342390
t_stop  = t_start+2000

xmax = xpos + size
xmin = xpos - size
ymax = ypos + size
ymin = ypos - size

sub = evt[(evt['X'] > xmin) & (evt['X'] < xmax) & (evt['Y'] > ymin) & (evt['Y'] < ymax)]
#sub = evt[(evt['RAWX'] > 21 - 3) & (evt['RAWX'] < 21+3) & (evt['RAWY'] > 181-3) & (evt['RAWY'] < 181+3) & evt['CCDNR'] == 1]
sub = sub[(sub['TIME'] > t_start) & (sub['TIME'] < t_stop)]
print(len(sub))

In [ ]:
tab_filt = sub
tab_filt = tab_filt[tab_filt['CCDNR'] == mode(tab_filt['CCDNR'])]
xbins = np.arange(tab_filt['RAWX'].min(), tab_filt['RAWX'].max(), 1)  
ybins = np.arange(tab_filt['RAWY'].min(), tab_filt['RAWY'].max(), 1)


In [ ]:
tab_filt

In [ ]:
im, _, _ = np.histogram2d(x=tab_filt['RAWX'], y=tab_filt['RAWY'], bins=[xbins,ybins])
plt.figure()
plt.imshow(im, aspect='equal', cmap='hot')
plt.show()

In [ ]:
sub.to_pandas()['RAWY'].value_counts()

In [ ]:
sub.to_pandas()['RAWX'].value_counts()

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(sub['TIME'], sub['PI']/1000, marker='.', s=10)

plt.xlim(t_start, t_stop)
plt.ylim(0,12)
plt.xlabel('Time (s)')
plt.ylabel('PI / 1000 (keV)')

In [ ]:
Energies = [[200, 12000],
            [200,2000],
            [2000,12000]]

colors = ['black', 'red', 'blue']

i=0
for E_lo, E_hi in Energies:
    sub2 = sub[(sub['PI'] > E_lo) & (sub['PI'] < E_hi)]
    x = np.arange(t_start, t_stop, 39)
    y = sub2['TIME']
    bins, edges = np.histogram(y, bins=x)
    bins = bins / 39
    plt.step(x[:-1], bins, label=f'{E_lo/1000} - {E_hi/1000} keV', color=colors[i])
    plt.ylabel('Count Rate (ct/s)')
    plt.xlabel('Time (s)')
    plt.legend()
    i+=1

In [ ]:
plt.figure()
plt.step(sub['t0'], sub['PI']/1000)
#plt.xlim(89810,89860)
#plt.ylim(0.14,0.3)
plt.xlabel('Time (s)')
plt.ylabel('PI / 1000 (keV)')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Create a figure with GridSpec to handle two panels
fig = plt.figure()
gs = gridspec.GridSpec(1, 2, width_ratios=[2.5, 1], wspace=0.0)

ax0 = fig.add_subplot(gs[0])
ax0.scatter(sub['t0'], sub['PI'] / 1000, marker='.', s=10, label='XMM PN 0.2-12.0 keV\nHydra A burst')
ax0.set_xlim(89815, 89855)
ax0.set_ylim(0.1, 0.3)
ax0.set_xlabel('Time (s)')
ax0.set_ylabel('Photon Energy (keV)')
leg = ax0.legend(loc='upper right')
leg.legend_handles[0].set_visible(False)

ax1 = fig.add_subplot(gs[1], sharey=ax0)
ax1.hist(sub['PI'] / 1000, bins=np.linspace(0.1, 0.3, 20), orientation='horizontal', histtype='step')
ax1.set_ylim(0.1, 0.3)
ax1.set_xlabel('Count')
ax1.yaxis.set_ticks_position("right")
plt.savefig('../../../data/plots/hydra_A_spec_lc.png')
plt.savefig('../../../data/plots/hydra_A_spec_lc.pdf')

plt.show()


In [ ]:
print(sub)

In [ ]:
xpos = 22933.122
ypos = 26315.909
size = 100
xmax = xpos + size
xmin = xpos - size
ymax = ypos + size
ymin = ypos - size

sub = evt[(evt['X'] > xmin) & (evt['X'] < xmax) & (evt['Y'] > ymin) & (evt['Y'] < ymax)]
sub['t0'] = sub['TIME'] - min(sub['TIME'])
plt.figure()
plt.scatter(sub['t0'], sub['RAWX'], marker='.', s=10)
plt.xlim(89815,89856)
#plt.ylim(181,183)
plt.xlabel('Time (s)')
plt.ylabel('PI / 1000 (keV)')


In [ ]:
89856 - 89815

In [ ]:
xpos = 22933.122
ypos = 26315.909
size = 100
xmax = xpos + size
xmin = xpos - size
ymax = ypos + size
ymin = ypos - size

sub = evt[(evt['X'] > xmin) & (evt['X'] < xmax) & (evt['Y'] > ymin) & (evt['Y'] < ymax)]
sub['t0'] = sub['TIME'] - min(sub['TIME'])
print(len(sub))
plt.figure()
plt.scatter(sub['t0'], sub['RAWY'], marker='.', s=10)
plt.xlim(89810,89860)
plt.ylim(181,183)
plt.xlabel('Time (s)')
plt.ylabel('PI / 1000 (keV)')


In [ ]:
tab_lc = Table.read('0843890301_PN0_dt1.lc')
tab_lc['t0'] = tab_lc['TIME'] - min(tab_lc['TIME'])
tab_lc

In [ ]:
sub = tab_lc[(tab_lc['t0'] > 90280) & (tab_lc['t0'] < 90350)]
#plt.step(sub['t0'], sub['RATE'], color='black')
plt.errorbar(sub['t0'], sub['RATE'], yerr=sub['ERROR'], ls='none', color='black', capsize=1.0, label='Hydra A Burst')
plt.errorbar(sub['t0'], sub['BACKV'], yerr=sub['BACKE'], ls='none', color='red', label='Background')
plt.ylim(0,15)
#plt.xlim(90200,90400)
plt.legend()

In [ ]:
evt = Table.read('pnfiltered.ds')
sub = evt[(evt['X'] > xmin) & (evt['X'] < xmax) & (evt['Y'] > ymin) & (evt['Y'] < ymax)]
sub['t0'] = sub['TIME'] - min(sub['TIME'])
sub = sub[(sub['t0'] > 89310) & (sub['t0'] < 92000)]
sub

In [ ]:
plt.figure(figsize=(10,10))
plt.scatter(sub['t0'], sub['PI']/1000)